### jupyter notebook to convert text to ids

author: Jan Jörg
date: 14.03.2024

In [53]:
# imports
import pandas as pd
from db import get_database

In [54]:
dbname = get_database()
collection = dbname["joined"]

documents = collection.find()
df = pd.DataFrame(list(documents))

df.head(2)

,_id,onetsoc19,display,occupation,tasks,technology_skills,tools_used,tools_technology,knowledge,skills,...,job_zone,interests,work_styles,work_values,related_occupations,additional_information,education,2010soc,isco08,Name_de
0,6620dfe3e366d70e7b9073fc,27-2011.00,long,"{'code': '27-2011.00', 'title': 'Actors', 'tag...","{'task': [{'id': 7646, 'green': False, 'relate...",{'category': [{'related': 'https://services.on...,{'category': [{'related': 'https://services.on...,{'tools': {'category': [{'related': 'https://s...,"{'element': [{'id': '2.C.7.c', 'related': 'htt...","{'element': [{'id': '2.A.1.a', 'related': 'htt...",...,"{'value': 2, 'title': 'Job Zone Two: Some Prep...","{'element': [{'id': '1.B.1.c', 'related': 'htt...","{'element': [{'id': '1.C.3.a', 'related': 'htt...","{'element': [{'id': '1.B.2.d', 'related': 'htt...",{'occupation': [{'href': 'https://services.one...,{'source': [{'url': 'https://www.actorsequity....,{'level_required': {'category': [{'name': 'Les...,27-2011,2655,Schauspieler
1,6620dfe3e366d70e7b9073fd,15-2011.00,long,"{'code': '15-2011.00', 'title': 'Actuaries', '...","{'task': [{'id': 3500, 'green': False, 'relate...",{'see_all': {'href': 'https://services.onetcen...,{'category': [{'related': 'https://services.on...,{'tools': {'category': [{'related': 'https://s...,"{'element': [{'id': '2.C.4.a', 'related': 'htt...","{'element': [{'id': '2.A.2.a', 'related': 'htt...",...,"{'value': 4, 'title': 'Job Zone Four: Consider...","{'element': [{'id': '1.B.1.f', 'related': 'htt...","{'element': [{'id': '1.C.7.b', 'related': 'htt...","{'element': [{'id': '1.B.2.b', 'related': 'htt...",{'occupation': [{'href': 'https://services.one...,{'source': [{'url': 'https://www.actuary.org/'...,"{'level_required': {'category': [{'name': ""Bac...",15-2011,2120,"Mathematiker, Aktuare und Statistiker"


In [55]:
skills = pd.DataFrame()
abilities = pd.DataFrame()
tasks = pd.DataFrame()
knowledge = pd.DataFrame()
work_activities = pd.DataFrame()
detailed_work_activities = pd.DataFrame()
work_context = pd.DataFrame()
job_zone = pd.DataFrame()
work_styles = pd.DataFrame()
work_values = pd.DataFrame()
education = pd.DataFrame()

In [56]:
# Assuming df["skills"] is a column of lists of dictionaries
skills = df['skills'].apply(lambda x: x['element']).explode().reset_index(drop=True)

# Convert the list of dictionaries into a DataFrame
skills_df = pd.json_normalize(skills)

# Drop duplicates based on 'id'
skills_df = skills_df.drop_duplicates(subset='id')

# Reset the index
skills_df = skills_df.reset_index(drop=True)

skills_df = skills_df.drop(columns=["related", "score.scale", "score.important", "score.value"])

skills_df['skill_id'] = range(1, len(skills_df) + 1)

skills_df.head(3)

,id,name,description,skill_id
0,2.A.1.a,Reading Comprehension,Understanding written sentences and paragraphs...,1
1,2.A.1.d,Speaking,Talking to others to convey information effect...,2
2,2.A.1.b,Active Listening,Giving full attention to what other people are...,3


In [57]:
dbname = get_database()

collection_name = dbname["skills"]

# Insert the dictionaries into the MongoDB collection
collection_name.insert_many(skills_df.to_dict("records"))

InsertManyResult([ObjectId('662553db621afd872127145b'), ObjectId('662553db621afd872127145c'), ObjectId('662553db621afd872127145d'), ObjectId('662553db621afd872127145e'), ObjectId('662553db621afd872127145f'), ObjectId('662553db621afd8721271460'), ObjectId('662553db621afd8721271461'), ObjectId('662553db621afd8721271462'), ObjectId('662553db621afd8721271463'), ObjectId('662553db621afd8721271464'), ObjectId('662553db621afd8721271465'), ObjectId('662553db621afd8721271466'), ObjectId('662553db621afd8721271467'), ObjectId('662553db621afd8721271468'), ObjectId('662553db621afd8721271469'), ObjectId('662553db621afd872127146a'), ObjectId('662553db621afd872127146b'), ObjectId('662553db621afd872127146c'), ObjectId('662553db621afd872127146d'), ObjectId('662553db621afd872127146e'), ObjectId('662553db621afd872127146f'), ObjectId('662553db621afd8721271470'), ObjectId('662553db621afd8721271471'), ObjectId('662553db621afd8721271472'), ObjectId('662553db621afd8721271473'), ObjectId('662553db621afd87212714

In [58]:
# Assuming df["abilities"] is a column of lists of dictionaries
abilities = df['abilities'].apply(lambda x: x['element']).explode().reset_index(drop=True)

# Convert the list of dictionaries into a DataFrame
abilities_df = pd.json_normalize(abilities)

# Drop duplicates based on 'id'
abilities_df = abilities_df.drop_duplicates(subset='id')

# Reset the index
abilities_df = abilities_df.reset_index(drop=True)

abilities_df = abilities_df.drop(columns=["related", "score.scale", "score.important" ,"score.value"])

abilities_df['ability_id'] = range(1, len(abilities_df) + 1)

abilities_df.head(3)

,id,name,description,ability_id
0,1.A.1.a.3,Oral Expression,The ability to communicate information and ide...,1
1,1.A.1.a.1,Oral Comprehension,The ability to listen to and understand inform...,2
2,1.A.1.d.1,Memorization,The ability to remember information such as wo...,3


In [59]:
dbname = get_database()

collection_name = dbname["abilities"]

# Insert the dictionaries into the MongoDB collection
collection_name.insert_many(abilities_df.to_dict("records"))

InsertManyResult([ObjectId('662553db621afd872127147f'), ObjectId('662553db621afd8721271480'), ObjectId('662553db621afd8721271481'), ObjectId('662553db621afd8721271482'), ObjectId('662553db621afd8721271483'), ObjectId('662553db621afd8721271484'), ObjectId('662553db621afd8721271485'), ObjectId('662553db621afd8721271486'), ObjectId('662553db621afd8721271487'), ObjectId('662553db621afd8721271488'), ObjectId('662553db621afd8721271489'), ObjectId('662553db621afd872127148a'), ObjectId('662553db621afd872127148b'), ObjectId('662553db621afd872127148c'), ObjectId('662553db621afd872127148d'), ObjectId('662553db621afd872127148e'), ObjectId('662553db621afd872127148f'), ObjectId('662553db621afd8721271490'), ObjectId('662553db621afd8721271491'), ObjectId('662553db621afd8721271492'), ObjectId('662553db621afd8721271493'), ObjectId('662553db621afd8721271494'), ObjectId('662553db621afd8721271495'), ObjectId('662553db621afd8721271496'), ObjectId('662553db621afd8721271497'), ObjectId('662553db621afd87212714

In [60]:
# Assuming df["tasks"] is a column of lists of dictionaries
tasks = df['tasks'].apply(lambda x: x['task']).explode().reset_index(drop=True)

# Convert the list of dictionaries into a DataFrame
tasks_df = pd.json_normalize(tasks)

# Drop duplicates based on 'id'
tasks_df = tasks_df.drop_duplicates(subset='id')

# Reset the index
tasks_df = tasks_df.reset_index(drop=True)

tasks_df = tasks_df.drop(columns=["green", "related","score.scale", "score.important", "score.value"])

tasks_df['task_id'] = range(1, len(tasks_df) + 1)

tasks_df.head(3)

,id,statement,category,task_id
0,7646,Collaborate with other actors as part of an en...,Core,1
1,7649,"Portray and interpret roles, using speech, ges...",Core,2
2,7644,"Work closely with directors, other actors, and...",Core,3


In [61]:
dbname = get_database()

collection_name = dbname["tasks"]

# Insert the dictionaries into the MongoDB collection
collection_name.insert_many(tasks_df.to_dict("records"))

InsertManyResult([ObjectId('662553db621afd87212714b4'), ObjectId('662553db621afd87212714b5'), ObjectId('662553db621afd87212714b6'), ObjectId('662553db621afd87212714b7'), ObjectId('662553db621afd87212714b8'), ObjectId('662553db621afd87212714b9'), ObjectId('662553db621afd87212714ba'), ObjectId('662553db621afd87212714bb'), ObjectId('662553db621afd87212714bc'), ObjectId('662553db621afd87212714bd'), ObjectId('662553db621afd87212714be'), ObjectId('662553db621afd87212714bf'), ObjectId('662553db621afd87212714c0'), ObjectId('662553db621afd87212714c1'), ObjectId('662553db621afd87212714c2'), ObjectId('662553db621afd87212714c3'), ObjectId('662553db621afd87212714c4'), ObjectId('662553db621afd87212714c5'), ObjectId('662553db621afd87212714c6'), ObjectId('662553db621afd87212714c7'), ObjectId('662553db621afd87212714c8'), ObjectId('662553db621afd87212714c9'), ObjectId('662553db621afd87212714ca'), ObjectId('662553db621afd87212714cb'), ObjectId('662553db621afd87212714cc'), ObjectId('662553db621afd87212714

In [62]:
# Assuming df["knowledge"] is a column of lists of dictionaries
knowledge = df['knowledge'].apply(lambda x: x['element']).explode().reset_index(drop=True)

# Convert the list of dictionaries into a DataFrame
knowledge_df = pd.json_normalize(knowledge)

# Drop duplicates based on 'id'
knowledge_df = knowledge_df.drop_duplicates(subset='id')

# Reset the index
knowledge_df = knowledge_df.reset_index(drop=True)

knowledge_df = knowledge_df.drop(columns=["related" ,"score.scale", "score.important", "score.value"])

knowledge_df['knowledge_id'] = range(1, len(knowledge_df) + 1)

knowledge_df.head(3)

,id,name,description,knowledge_id
0,2.C.7.c,Fine Arts,Knowledge of the theory and techniques require...,1
1,2.C.7.a,English Language,Knowledge of the structure and content of the ...,2
2,2.C.9.b,Communications and Media,"Knowledge of media production, communication, ...",3


In [63]:
dbname = get_database()

collection_name = dbname["knowledge"]

# Insert the dictionaries into the MongoDB collection
collection_name.insert_many(knowledge_df.to_dict("records"))

InsertManyResult([ObjectId('662553e0621afd87212747ae'), ObjectId('662553e0621afd87212747af'), ObjectId('662553e0621afd87212747b0'), ObjectId('662553e0621afd87212747b1'), ObjectId('662553e0621afd87212747b2'), ObjectId('662553e0621afd87212747b3'), ObjectId('662553e0621afd87212747b4'), ObjectId('662553e0621afd87212747b5'), ObjectId('662553e0621afd87212747b6'), ObjectId('662553e0621afd87212747b7'), ObjectId('662553e0621afd87212747b8'), ObjectId('662553e0621afd87212747b9'), ObjectId('662553e0621afd87212747ba'), ObjectId('662553e0621afd87212747bb'), ObjectId('662553e0621afd87212747bc'), ObjectId('662553e0621afd87212747bd'), ObjectId('662553e0621afd87212747be'), ObjectId('662553e0621afd87212747bf'), ObjectId('662553e0621afd87212747c0'), ObjectId('662553e0621afd87212747c1'), ObjectId('662553e0621afd87212747c2'), ObjectId('662553e0621afd87212747c3'), ObjectId('662553e0621afd87212747c4'), ObjectId('662553e0621afd87212747c5'), ObjectId('662553e0621afd87212747c6'), ObjectId('662553e0621afd87212747

In [64]:
# Assuming df["work_activities"] is a column of lists of dictionaries
work_activities = df['work_activities'].apply(lambda x: x['element']).explode().reset_index(drop=True)

# Convert the list of dictionaries into a DataFrame
work_activities_df = pd.json_normalize(work_activities)

# Drop duplicates based on 'id'
work_activities_df = work_activities_df.drop_duplicates(subset='id')

# Reset the index
work_activities_df = work_activities_df.reset_index(drop=True)

work_activities_df = work_activities_df.drop(columns=["related" ,"score.scale", "score.important", "score.value"])

work_activities_df['work_activity_id'] = range(1, len(work_activities_df) + 1)

work_activities_df.head(3)

,id,name,description,work_activity_id
0,4.A.4.a.4,Establishing and Maintaining Interpersonal Rel...,Developing constructive and cooperative workin...,1
1,4.A.2.b.2,Thinking Creatively,"Developing, designing, or creating new applica...",2
2,4.A.4.a.8,Performing for or Working Directly with the Pu...,Performing for people or dealing directly with...,3


In [65]:
dbname = get_database()

collection_name = dbname["work_activities"]

# Insert the dictionaries into the MongoDB collection
collection_name.insert_many(work_activities_df.to_dict("records"))

InsertManyResult([ObjectId('662553e3621afd87212747d0'), ObjectId('662553e3621afd87212747d1'), ObjectId('662553e3621afd87212747d2'), ObjectId('662553e3621afd87212747d3'), ObjectId('662553e3621afd87212747d4'), ObjectId('662553e3621afd87212747d5'), ObjectId('662553e3621afd87212747d6'), ObjectId('662553e3621afd87212747d7'), ObjectId('662553e3621afd87212747d8'), ObjectId('662553e3621afd87212747d9'), ObjectId('662553e3621afd87212747da'), ObjectId('662553e3621afd87212747db'), ObjectId('662553e3621afd87212747dc'), ObjectId('662553e3621afd87212747dd'), ObjectId('662553e3621afd87212747de'), ObjectId('662553e3621afd87212747df'), ObjectId('662553e3621afd87212747e0'), ObjectId('662553e3621afd87212747e1'), ObjectId('662553e3621afd87212747e2'), ObjectId('662553e3621afd87212747e3'), ObjectId('662553e3621afd87212747e4'), ObjectId('662553e3621afd87212747e5'), ObjectId('662553e3621afd87212747e6'), ObjectId('662553e3621afd87212747e7'), ObjectId('662553e3621afd87212747e8'), ObjectId('662553e3621afd87212747

In [66]:
# Assuming df["detailed_work_activities"] is a column of lists of dictionaries
detailed_work_activities = df['detailed_work_activities'].apply(lambda x: x['activity']).explode().reset_index(drop=True)

# Convert the list of dictionaries into a DataFrame
detailed_work_activities_df = pd.json_normalize(detailed_work_activities)

# Drop duplicates based on 'id'
detailed_work_activities_df = detailed_work_activities_df.drop_duplicates(subset='id')

# Reset the index
detailed_work_activities_df = detailed_work_activities_df.reset_index(drop=True)

detailed_work_activities_df = detailed_work_activities_df.drop(columns=["related"])

detailed_work_activities_df['detailed_work_activity_id'] = range(1, len(detailed_work_activities_df) + 1)

detailed_work_activities_df.head(3)

,id,name,detailed_work_activity_id
0,4.A.4.a.8.I01.D04,Collaborate with others to prepare or perform ...,1
1,4.A.4.a.8.I01.D05,Entertain public with comedic or dramatic perf...,2
2,4.A.1.a.1.I01.D04,Study scripts to determine project requirements.,3


In [67]:
dbname = get_database()

collection_name = dbname["detailed_work_activities"]

# Insert the dictionaries into the MongoDB collection
collection_name.insert_many(detailed_work_activities_df.to_dict("records"))

InsertManyResult([ObjectId('662553e4621afd87212747fa'), ObjectId('662553e4621afd87212747fb'), ObjectId('662553e4621afd87212747fc'), ObjectId('662553e4621afd87212747fd'), ObjectId('662553e4621afd87212747fe'), ObjectId('662553e4621afd87212747ff'), ObjectId('662553e4621afd8721274800'), ObjectId('662553e4621afd8721274801'), ObjectId('662553e4621afd8721274802'), ObjectId('662553e4621afd8721274803'), ObjectId('662553e4621afd8721274804'), ObjectId('662553e4621afd8721274805'), ObjectId('662553e4621afd8721274806'), ObjectId('662553e4621afd8721274807'), ObjectId('662553e4621afd8721274808'), ObjectId('662553e4621afd8721274809'), ObjectId('662553e4621afd872127480a'), ObjectId('662553e4621afd872127480b'), ObjectId('662553e4621afd872127480c'), ObjectId('662553e4621afd872127480d'), ObjectId('662553e4621afd872127480e'), ObjectId('662553e4621afd872127480f'), ObjectId('662553e4621afd8721274810'), ObjectId('662553e4621afd8721274811'), ObjectId('662553e4621afd8721274812'), ObjectId('662553e4621afd87212748

In [68]:
# Assuming df["work_context"] is a column of lists of dictionaries
work_context = df['work_context'].apply(lambda x: x['element']).explode().reset_index(drop=True)

# Convert the list of dictionaries into a DataFrame
work_context_df = pd.json_normalize(work_context)

# Drop duplicates based on 'id'
work_context_df = work_context_df.drop_duplicates(subset='id')

# Reset the index
work_context_df = work_context_df.reset_index(drop=True)

work_context_df = work_context_df.drop(columns=["related", "response", "score.scale", "score.important", "score.value"])

work_context_df['work_context_id'] = range(1, len(work_context_df) + 1)

work_context_df.head(3)

,id,name,description,work_context_id
0,4.C.1.b.1.e,Work With Work Group or Team,How important is it to work with others in a g...,1
1,4.C.2.a.3,Physical Proximity,To what extent does this job require the worke...,2
2,4.C.1.a.4,Contact With Others,How much does this job require the worker to b...,3


In [69]:
dbname = get_database()

collection_name = dbname["work_context"]

# Insert the dictionaries into the MongoDB collection
collection_name.insert_many(work_context_df.to_dict("records"))

InsertManyResult([ObjectId('662553e7621afd8721274fd2'), ObjectId('662553e7621afd8721274fd3'), ObjectId('662553e7621afd8721274fd4'), ObjectId('662553e7621afd8721274fd5'), ObjectId('662553e7621afd8721274fd6'), ObjectId('662553e7621afd8721274fd7'), ObjectId('662553e7621afd8721274fd8'), ObjectId('662553e7621afd8721274fd9'), ObjectId('662553e7621afd8721274fda'), ObjectId('662553e7621afd8721274fdb'), ObjectId('662553e7621afd8721274fdc'), ObjectId('662553e7621afd8721274fdd'), ObjectId('662553e7621afd8721274fde'), ObjectId('662553e7621afd8721274fdf'), ObjectId('662553e7621afd8721274fe0'), ObjectId('662553e7621afd8721274fe1'), ObjectId('662553e7621afd8721274fe2'), ObjectId('662553e7621afd8721274fe3'), ObjectId('662553e7621afd8721274fe4'), ObjectId('662553e7621afd8721274fe5'), ObjectId('662553e7621afd8721274fe6'), ObjectId('662553e7621afd8721274fe7'), ObjectId('662553e7621afd8721274fe8'), ObjectId('662553e7621afd8721274fe9'), ObjectId('662553e7621afd8721274fea'), ObjectId('662553e7621afd8721274f

In [70]:
# Assuming df["work_styles"] is a column of lists of dictionaries
work_styles = df['work_styles'].apply(lambda x: x['element']).explode().reset_index(drop=True)

# Convert the list of dictionaries into a DataFrame
work_styles_df = pd.json_normalize(work_styles)

# Drop duplicates based on 'id'
work_styles_df = work_styles_df.drop_duplicates(subset='id')

# Reset the index
work_styles_df = work_styles_df.reset_index(drop=True)

work_styles_df = work_styles_df.drop(columns=["related", "score.scale", "score.important", "score.value"])

work_styles_df['work_style_id'] = range(1, len(work_styles_df) + 1)

work_styles_df.head(3)

,id,name,description,work_style_id
0,1.C.3.a,Cooperation,Job requires being pleasant with others on the...,1
1,1.C.1.b,Persistence,Job requires persistence in the face of obstac...,2
2,1.C.4.c,Adaptability/Flexibility,Job requires being open to change (positive or...,3


In [71]:
dbname = get_database()

collection_name = dbname["work_styles"]

# Insert the dictionaries into the MongoDB collection
collection_name.insert_many(work_styles_df.to_dict("records"))

InsertManyResult([ObjectId('662553e8621afd872127500c'), ObjectId('662553e8621afd872127500d'), ObjectId('662553e8621afd872127500e'), ObjectId('662553e8621afd872127500f'), ObjectId('662553e8621afd8721275010'), ObjectId('662553e8621afd8721275011'), ObjectId('662553e8621afd8721275012'), ObjectId('662553e8621afd8721275013'), ObjectId('662553e8621afd8721275014'), ObjectId('662553e8621afd8721275015'), ObjectId('662553e8621afd8721275016'), ObjectId('662553e8621afd8721275017'), ObjectId('662553e8621afd8721275018'), ObjectId('662553e8621afd8721275019'), ObjectId('662553e8621afd872127501a'), ObjectId('662553e8621afd872127501b')], acknowledged=True)

In [72]:
# Assuming df["work_values"] is a column of lists of dictionaries
work_values = df['work_values'].apply(lambda x: x['element']).explode().reset_index(drop=True)

# Convert the list of dictionaries into a DataFrame
work_values_df = pd.json_normalize(work_values)

# Drop duplicates based on 'id'
work_values_df = work_values_df.drop_duplicates(subset='id')

# Reset the index
work_values_df = work_values_df.reset_index(drop=True)

work_values_df = work_values_df.drop(columns=["related", "score.scale", "score.important", "score.value"])

work_values_df['work_value_id'] = range(1, len(work_values_df) + 1)

work_values_df.head(3)

,id,name,description,work_value_id
0,1.B.2.d,Relationships,Occupations that satisfy this work value allow...,1
1,1.B.2.a,Achievement,Occupations that satisfy this work value are r...,2
2,1.B.2.f,Independence,Occupations that satisfy this work value allow...,3


In [73]:
dbname = get_database()

collection_name = dbname["work_values"]

# Insert the dictionaries into the MongoDB collection
collection_name.insert_many(work_values_df.to_dict("records"))

InsertManyResult([ObjectId('662553e8621afd872127501d'), ObjectId('662553e8621afd872127501e'), ObjectId('662553e8621afd872127501f'), ObjectId('662553e8621afd8721275020'), ObjectId('662553e8621afd8721275021'), ObjectId('662553e8621afd8721275022')], acknowledged=True)

In [74]:
job_zone = pd.DataFrame(list(df["job_zone"]))

# Drop duplicates based on 'value'
job_zone = job_zone.drop_duplicates(subset='value')
job_zone = job_zone.drop(columns=["svp_range", "related_experience"])

job_zone['job_zone_id'] = range(1, len(job_zone) + 1)

job_zone.head(2)

,value,title,education,job_training,job_zone_examples,job_zone_id
0,2,Job Zone Two: Some Preparation Needed,These occupations usually require a high schoo...,Employees in these occupations need anywhere f...,These occupations often involve using your kno...,1
1,4,Job Zone Four: Considerable Preparation Needed,Most of these occupations require a four-year ...,Employees in these occupations usually need se...,Many of these occupations involve coordinating...,2


In [75]:
dbname = get_database()

collection_name = dbname["job_zone"]

# Insert the dictionaries into the MongoDB collection
collection_name.insert_many(job_zone.to_dict("records"))

InsertManyResult([ObjectId('662553e9621afd8721275024'), ObjectId('662553e9621afd8721275025'), ObjectId('662553e9621afd8721275026'), ObjectId('662553e9621afd8721275027'), ObjectId('662553e9621afd8721275028')], acknowledged=True)

In [76]:
# Assuming df["work_values"] is a column of lists of dictionaries
education = df['education'].apply(lambda x: x['level_required']).apply(lambda x : x["category"]).explode().reset_index(drop=True)

# Convert the list of dictionaries into a DataFrame
education_df = pd.json_normalize(education)

# Drop duplicates based on 'name'
education_df = education_df.drop_duplicates(subset='name')
education_df = education_df.drop(columns=["score.scale", "score.value", "description"])

education_df['education_id'] = range(1, len(education_df) + 1)

education_df.head(5)

,name,education_id
0,Less than high school diploma,1
1,Bachelor's degree,2
2,High school diploma or equivalent,3
4,Post-baccalaureate certificate,4
5,Professional degree,5


In [77]:
dbname = get_database()

collection_name = dbname["education"]

# Insert the dictionaries into the MongoDB collection
collection_name.insert_many(education_df.to_dict("records"))

InsertManyResult([ObjectId('662553e9621afd872127502a'), ObjectId('662553e9621afd872127502b'), ObjectId('662553e9621afd872127502c'), ObjectId('662553e9621afd872127502d'), ObjectId('662553e9621afd872127502e'), ObjectId('662553e9621afd872127502f'), ObjectId('662553e9621afd8721275030'), ObjectId('662553e9621afd8721275031'), ObjectId('662553e9621afd8721275032'), ObjectId('662553e9621afd8721275033'), ObjectId('662553e9621afd8721275034'), ObjectId('662553e9621afd8721275035')], acknowledged=True)